# Excercises 
# 1. Tune the network
Run the experiment below, explore the different parameters (see suggestions below) and study the result with tensorboard. 
Make a single page (1 a4) report of your findings. Use your visualisation skills to communicate your most important findings.

In [15]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [16]:
%tensorboard --logdir modellogs/

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\Alex\AppData\Local\Programs\Python\Python312\Scripts\tensorboard.exe\__main__.py", line 2, in <module>
  File "C:\Users\Alex\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorboard\main.py", line 27, in <module>
    from tensorboard import default
  File "C:\Users\Alex\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorboard\default.py", line 30, in <module>
    import pkg_resources
ModuleNotFoundError: No module named 'pkg_resources'

In [17]:
import sys
print(sys.executable)

c:\Users\Alex\AppData\Local\Programs\Python\Python311\python.exe


In [18]:
from mads_datasets import DatasetFactoryProvider, DatasetType

from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics

import torch.optim as optim
from torch import nn
from tomlserializer import TOMLSerializer

We will be using `tomlserializer` to easily keep track of our experiments, and to easily save the different things we did during our experiments.
It can export things like settings and models to a simple `toml` file, which can be easily shared, checked and modified.

First, we need the data. 

In [19]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2026-02-26 18:28:08.653 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\Alex\.cache\mads_datasets\fashionmnist
2026-02-26 18:28:08.654 | INFO     | mads_datasets.base:download_data:124 - File already exists at C:\Users\Alex\.cache\mads_datasets\fashionmnist\fashionmnist.pt


We need a way to determine how well our model is performing. We will use accuracy as a metric.

In [20]:
accuracy = metrics.Accuracy()

You can set up a single experiment.

- We will show the model batches of 64 images, 
- and for every epoch we will show the model 100 batches (trainsteps=100).
- then, we will test how well the model is doing on unseen data (teststeps=100).
- we will report our results during training to tensorboard, and report all configuration to a toml file.
- we will log the results into a directory called "modellogs", but you could change this to whatever you want.

In [21]:
import torch
loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellogs",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)


We will use a very basic model: a model with three linear layers.

In [48]:
import torch
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, units1),
            nn.BatchNorm1d(units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            # nn.Dropout(0.5),
            nn.Linear(units2, num_classes)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(num_classes=10, units1=256, units2=128)

I developped the `tomlserializer` package, it is a useful tool to save configs, models and settings as a tomlfile; that way it is easy to track what you changed during your experiments.

This package will 1. check if there is a `__dict__` attribute available, and if so, it will use that to extract the parameters that do not start with an underscore, like this:

In [23]:
{k: v for k, v in model.__dict__.items() if not k.startswith("_")}

{'training': True, 'num_classes': 10, 'units1': 256, 'units2': 256}

This means that if you want to add more parameters to the `.toml` file, eg `units3`, you can add them to the class like this:

```python
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes: int, units1: int, units2: int, units3: int) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.units1 = units1
        self.units2 = units2
        self.units3 = units3  # <-- add this line
```

And then it will be added to the `.toml` file. Check the result for yourself by using the `.save()` method of the `TomlSerializer` class like this:

In [24]:
tomlserializer = TOMLSerializer()
tomlserializer.save(settings, "settings.toml")
tomlserializer.save(model, "model.toml")

Check the `settings.toml` and `model.toml` files to see what is in there.

You can use the `Trainer` class from my `mltrainer` module to train your model. It has the TOMLserializer integrated, so it will automatically save the settings and model to a toml file if you have added `TOML` as a reporttype in the settings.

In [25]:
trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
)
trainer.loop()

2026-02-26 18:28:08.730 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\20260226-182808
2026-02-26 18:28:08.731 | INFO     | mltrainer.trainer:__init__:66 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 100/100 [00:00<00:00, 188.34it/s]
2026-02-26 18:28:09.555 | INFO     | mltrainer.trainer:report:215 - Epoch 0 train 0.9155 test 0.6590 metric ['0.7636']
100%|██████████| 100/100 [00:00<00:00, 197.38it/s]
2026-02-26 18:28:10.346 | INFO     | mltrainer.trainer:report:215 - Epoch 1 train 0.5633 test 0.5641 metric ['0.7889']
100%|██████████| 100/100 [00:00<00:00, 184.17it/s]
2026-02-26 18:28:11.167 | INFO     | mltrainer.trainer:report:215 - Epoch 2 train 0.5215 test 0.5153 metric ['0.8133']
100%|██████████| 3/3 [00:02<00:00,  1.23it/s]


Now, check in the modellogs directory the results of your experiment.

We can now loop this with a naive approach, called a grid-search (why do you think i call it naive?).

In [26]:
units = [256, 128, 64]
for unit1 in units:
    for unit2 in units:
        print(f"Units: {unit1}, {unit2}")

Units: 256, 256
Units: 256, 128
Units: 256, 64
Units: 128, 256
Units: 128, 128
Units: 128, 64
Units: 64, 256
Units: 64, 128
Units: 64, 64


Of course, this might not be the best way to search for a model; some configurations will be better than others (can you predict up front what will be the best configuration?).

So, feel free to improve upon the gridsearch by adding your own logic.

In [49]:
import torch

# Baseline configuratie
baseline = {
    "units1": 256,
    "units2": 128,
    "epochs": 10,
    "batchsize": 32,
    "lr": 0.001
}

print(f"Running BASELINE experiment: {baseline}")

# Model
model = NeuralNetwork(
    num_classes=10,
    units1=baseline["units1"],
    units2=baseline["units2"],
    #dropout_rate = 0.1
)

# Trainer settings
settings = TrainerSettings(
    epochs=baseline["epochs"],
    metrics=[accuracy],
    logdir="modellogs/Normalization",
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
    learningrate=baseline["lr"]
)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=baseline["lr"])

# Trainer
trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=torch.optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=None
)

# Start training
trainer.loop()

2026-02-26 19:21:11.135 | INFO     | mltrainer.settings:check_path:60 - Created logdir c:\Users\Alex\Documents\GitHub\portfolio-example\2-hypertuning-mlflow\modellogs\Normalization
2026-02-26 19:21:11.136 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs\Normalization\20260226-192111
2026-02-26 19:21:11.137 | INFO     | mltrainer.trainer:__init__:66 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.


Running BASELINE experiment: {'units1': 256, 'units2': 128, 'epochs': 10, 'batchsize': 32, 'lr': 0.001}


100%|██████████| 937/937 [00:05<00:00, 185.54it/s]
2026-02-26 19:21:16.646 | INFO     | mltrainer.trainer:report:215 - Epoch 0 train 0.4524 test 0.4056 metric ['0.8511']
100%|██████████| 937/937 [00:04<00:00, 193.40it/s]
2026-02-26 19:21:21.996 | INFO     | mltrainer.trainer:report:215 - Epoch 1 train 0.3437 test 0.3953 metric ['0.8520']
100%|██████████| 937/937 [00:04<00:00, 194.44it/s]
2026-02-26 19:21:27.224 | INFO     | mltrainer.trainer:report:215 - Epoch 2 train 0.3098 test 0.3948 metric ['0.8572']
100%|██████████| 937/937 [00:04<00:00, 216.35it/s]
2026-02-26 19:21:31.967 | INFO     | mltrainer.trainer:report:215 - Epoch 3 train 0.2819 test 0.3542 metric ['0.8710']
100%|██████████| 937/937 [00:04<00:00, 195.83it/s]
2026-02-26 19:21:37.172 | INFO     | mltrainer.trainer:report:215 - Epoch 4 train 0.2653 test 0.3545 metric ['0.8766']
2026-02-26 19:21:37.172 | INFO     | mltrainer.trainer:__call__:258 - best loss: 0.3542, current loss 0.3545.Counter 1/10.
100%|██████████| 937/937 [0

Because we have set the ReportType to TOML, you will find in every log dir a model.toml and settings.toml file.

Run the experiment, and study the result with tensorboard. 

Locally, it is easy to do that with VS code itself. On the server, you have to take these steps:

- in the terminal, `cd` to the location of the repository
- activate the python environment for the shell. Note how the correct environment is being activated.
- run `tensorboard --logdir=modellogs` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`